# Lecture 4 -- dynamics of the rimless wheel

In this notebook we will analyze the steady state dynamics of the rimless wheel.

In [15]:
import numpy as np
from scipy.integrate import solve_ivp

import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()


import warnings
warnings.filterwarnings('ignore')

Let's define a helper function to convert between rad->deg

In [16]:
def deg2rad(deg):
    return deg*np.pi/180

## Define the initial conditions

We will use these to first derive our simulation of the wheel dynamics. The rimless wheel parameters are shown in the diagram below. 

![](rimless_wheel.png)

In [17]:
# System constants
L = 1 
alpha = deg2rad(20)
gamma = deg2rad(30)
gravity = 9.8

# Initial condition
theta = 0
dtheta_dt = .1

dt = 0.001
time = np.linspace(0,10,10/dt)

## Define the equations of motions

The equations of motion of the wheel are just that of an inverted pendulum, with the caveat that when the wheel angle reaches $\alpha + \gamma$ the next stance phase is initiated. So let's also define a step_event to detect collisions, and a reset function to reset the state variables to the next stance.

In [18]:
# General inverted pendulum simulator
def rimless_wheel(t, state, l):
    theta, omega = state
    dydt = [omega, (gravity/l)*np.sin(theta)]
    return dydt

# Detect a step
def step_event(t, state):
    theta, omega = state
    return theta - (alpha + gamma) # when this is zero a collision occurred
    
step_event.terminal = True

# The wheel has a specific reset condition where energy is taken out of the 
# the system v_new = v_old*cos(2*alpha)
def collision_reset(state):
    theta, omega = state
    
    theta = gamma - alpha
    omega = omega * np.cos(2*alpha)
    
    return [theta, omega]



Check that we can detect the foot touch-down event

In [19]:
sol = solve_ivp(lambda t, y: rimless_wheel(t, y, L), 
                [0,10], [theta, 1], 
                t_eval = time,
                events = step_event)

plt.clf()
plt.plot(sol.t, sol.y[0,:])
plt.ylabel('$\\theta$')
plt.xlabel('time')
plt.gcf()

<Figure size 1280x960 with 1 Axes>

Cool, so the above plot shows the wheel rolling down the hill until the next spoke hits the ground. Then, as we specified in the solver the integration stops. This is simulating one step!


Now let's simulate two steps with a foot reset

In [20]:
t_data = []
y1 = []
y2 = []

sol = solve_ivp(lambda t, y: rimless_wheel(t, y, L), 
                [0,10], [theta, 1], 
                t_eval = time,
                events = step_event)

end_step = sol.t.shape[0]
reset_state = collision_reset(sol.y[:,-1])
t_data.append(sol.t)
y1.append(sol.y[0,:])
y2.append(sol.y[1,:])


sol = solve_ivp(lambda t, y: rimless_wheel(t, y, L), 
                [time[end_step],10], reset_state, 
                t_eval = time[end_step:],
                events = step_event)

end_step = sol.t.shape[0]
reset_state = collision_reset(sol.y[:,-1])
t_data.append(sol.t)
y1.append(sol.y[0,:])
y2.append(sol.y[1,:])

plt.clf()
plt.plot(t_data[0], y1[0])
plt.plot(t_data[1], y1[1])
plt.ylabel('$\\theta$')
plt.xlabel('time')
plt.gcf()

<Figure size 640x480 with 1 Axes>

Cool again, now we see two steps, both ending at the same $\theta$ but with a slightly different shape to them. 

Now lets simulate 20 steps!

In [21]:
# initialize data storage lists
t_data = []
y1 = []
y2 = []

# this is the starting state
reset_state = [0, 0.0005]
t0 = 0

# Loop through 20 steps
for kk in range(20):
    sol = solve_ivp(lambda t, y: rimless_wheel(t, y, L), 
                [t0,10], reset_state, 
                t_eval = np.linspace(t0+dt, 10, (10-(t0-dt))/dt),
                events = step_event)
    
    t0 = sol.t[-1]
    reset_state = collision_reset(sol.y[:,-1])
    t_data.append(sol.t)
    y1.append(sol.y[0,:])
    y2.append(sol.y[1,:])
    
    if t0 == 10.:
        break

plt.clf()

for tt, yy in zip(t_data, y1):
    plt.plot(tt, yy)

plt.ylabel('$\\theta$')
plt.xlabel('time')
plt.gcf()


<Figure size 640x480 with 1 Axes>

Very cool. We notice something about this system, the initially are different but start to approach a similar shape.

We can visualize this in a different way by plotting the phase space $(x, y) = (\theta, \dot{\theta})$

In [22]:
t_data = []
y1 = []
y2 = []

# this is the starting state
reset_state = [deg2rad(0), 0.15]
t0 = 0
end_time = 20

gamma = np.pi/10
alpha = np.pi/5.5

# Loop through 20 steps
for kk in range(20):
    sol = solve_ivp(lambda t, y: rimless_wheel(t, y, L), 
                [t0,end_time], reset_state, 
                t_eval = np.linspace(t0+dt, end_time, (end_time-(t0-dt))/dt),
                events = step_event)
    
    t0 = sol.t[-1]
    reset_state = collision_reset(sol.y[:,-1])
    t_data.append(sol.t)
    y1.append(sol.y[0,:])
    y2.append(sol.y[1,:])
    
    if t0 == end_time:
        break

plt.clf()
plt.plot([-2,2],[0,0], color = [.9, .9, .9])
plt.plot([0,0],[-20,20], color = [.9, .9, .9])
for yy1, yy2 in zip(y1, y2):
    plt.plot(yy1, yy2)

plt.axis([-.7, .7, -3, 3])
plt.xlabel('$\\theta$')
plt.ylabel('$\dot{\\theta}$')
plt.gcf()

<Figure size 640x480 with 1 Axes>

To watch the evolution of this system let's animate the wheel and the evolution through phase space. I have included a .gif below of what you should see. 

In [ ]:
def animate_rimless_wheel(timeseries_list, theta_series_list, theta_dot_series_list, gamma, alpha, skip = 1):
    
    theta_dot_series = np.hstack(theta_dot_series_list)
    theta_series = np.hstack(theta_series_list)
    timeseries = np.hstack(timeseries_list)
    
    timeseries = timeseries[0::skip]
    theta_series = theta_series[0::skip]
    theta_dot_series = theta_dot_series[0::skip]

    plt.clf()
    plt.subplot(1,2,1)
    plt.subplot(1,2,2)

    for yy1, yy2 in zip(theta_series_list, theta_dot_series_list):
        plt.plot(yy1, yy2, color=[0.7, 0.7, 0.7])
    
    marker, = plt.plot(theta_series[0], theta_dot_series[0], 'ro')
    plt.axis([-1, 1, -5, 5])
    plt.xlabel('$\\theta$')
    plt.ylabel('$\dot{\\theta}$')

    left_or_right = 0
    
    for kk, (tt, q, q_dot) in enumerate(zip(timeseries, theta_series, theta_dot_series)):
        plt.subplot(1,2,1)
        plt.cla()
        plt.plot(np.array([0, np.cos(-q + np.pi/2)]) - np.cos(-q + np.pi/2), \
                 np.array([0, np.sin(-q + np.pi/2)]) - np.sin(-q + np.pi/2), 'ko-')
        plt.plot(np.array([0, np.cos(-(q-2*alpha) + np.pi/2)]) - np.cos(-(q-2*alpha) + np.pi/2), \
                 np.array([0, np.sin(-(q-2*alpha) + np.pi/2)]) - np.sin(-(q-2*alpha) + np.pi/2), 'ko-')

        plt.plot(np.array([-10*np.cos(gamma), 10*np.cos(gamma)]) - np.cos(-q + np.pi/2), 
                 np.array([10*np.sin(gamma), -10*np.sin(gamma)]) - np.sin(-q + np.pi/2))
                  
        plt.axis([-2,2,-2,2])
                  
                  
        plt.subplot(1,2,2)
        marker.set_xdata(q)
        marker.set_ydata(q_dot)
    
        plt.title(tt)
        
        plt.pause(0.001)
        
        # If you want to save to make a gif
#         plt.savefig("{:05}".format(kk) + ".png")


animate_rimless_wheel(t_data, y1, y2, gamma, alpha, skip = 20)
    
    
    

Let's also make a function to integrate the dynamics easilu

In [ ]:
def simulate_rimless(initial_state, gamma, alpha, t0 = 0, end_time = 20):

    t_data = []
    y1 = []
    y2 = []
    reset_state = initial_state

    # Loop through 20 steps
    for kk in range(20):
        sol = solve_ivp(lambda t, y: rimless_wheel(t, y, L), 
                    [t0,end_time], reset_state, 
                    t_eval = np.linspace(t0+dt, end_time, (end_time-(t0-dt))/dt),
                    events = step_event)

        t0 = sol.t[-1]
        reset_state = collision_reset(sol.y[:,-1])
        t_data.append(sol.t)
        y1.append(sol.y[0,:])
        y2.append(sol.y[1,:])

        if t0 == end_time:
            break

    return t_data, y1, y2
    

gamma = np.pi/12
alpha = np.pi/6
initial = [deg2rad(0), 4.15]

t_data, y1, y2 = simulate_rimless(initial, gamma, alpha)

animate_rimless_wheel(t_data, y1, y2, gamma, alpha, skip = 20)

        

Let's try some different parameters

In [26]:
# gamma = np.pi/12
# alpha = np.pi/6

gamma = np.pi/10
alpha = np.pi/10.5

initial = [deg2rad(0), 5]

t_data, y1, y2 = simulate_rimless(initial, gamma, alpha)

animate_rimless_wheel(t_data, y1, y2, gamma, alpha, skip = 20)

In [27]:
gamma = np.pi/7
alpha = np.pi/6
initial = [deg2rad(0), 40.15]

t_data, y1, y2 = simulate_rimless(initial, gamma, alpha)

animate_rimless_wheel(t_data, y1, y2, gamma, alpha, skip = 20)



In [28]:
gamma = np.pi/12
alpha = np.pi/10
initial = [deg2rad(0.4), 0]

t_data, y1, y2 = simulate_rimless(initial, gamma, alpha)

animate_rimless_wheel(t_data, y1, y2, gamma, alpha, skip = 20)



Ok, now let's map out the phase space for a given alpha and gamma at different initial conditions

In [30]:

gamma = np.pi/12
alpha = np.pi/6

random_state = np.array([0.6, 5])*(np.random.rand(100,2))
plt.clf()
dk = 0.9
alpha_viz = 0.2

plt.figure()

for rnd in random_state:
#     print(rnd)
    t_data, y1, y2 = simulate_rimless(rnd, gamma, alpha)
    plt.plot([-2,2],[0,0], color = [.9, .9, .9])
    plt.plot([0,0],[-20,20], color = [.9, .9, .9])
    for yy1, yy2 in zip(y1, y2):
        plt.plot(yy1, yy2, color = [dk, dk, dk, alpha_viz])

plt.axis([-1, 1, -5, 5])
plt.xlabel('$\\theta$')
plt.ylabel('$\dot{\\theta}$')
plt.gcf()

<Figure size 640x480 with 1 Axes>

### Now let's compute the return map

For this we only need to keep the previous touchdown velocity, and the new touchdown velocity

In [31]:
gamma = np.pi/12
alpha = np.pi/6

random_state = np.array([1, 5])*(np.random.rand(100,2))
plt.clf()
dk = 0.9
alpha_viz = 0.1

theta_dot_at_touchdown = []

for rnd in random_state:
    t_data, y1, y2 = simulate_rimless(rnd, gamma, alpha)
    theta_dot_at_touchdown.append([y2[k][0] for k in range(1,len(y2))])

for t_touchdown in theta_dot_at_touchdown:
    x = t_touchdown[0:-2]
    y = t_touchdown[1:-1]
    plt.plot(x,y, '.')
    
plt.plot([0,3],[0,3])

plt.xlabel('$\dot{\\theta}_{n}^+$')
plt.ylabel('$\dot{\\theta}_{n+1}^+$')
plt.gcf()

<Figure size 640x480 with 1 Axes>

This is a little hard to interpret, so let's animate the return map

In [33]:
gamma = np.pi/12
alpha = np.pi/5.5

random_state = np.array([1, 5])*(np.random.rand(10,2))

random_state[0,:] = [gamma + alpha-0.03, 0]

plt.clf()
dk = 0.9
alpha_viz = 0.1

theta_dot_at_touchdown = []

for rnd in random_state:
    t_data, y1, y2 = simulate_rimless(rnd, gamma, alpha)
    theta_dot_at_touchdown.append([y2[k][0] for k in range(1,len(y2))])
    
dk = 0.9
tstep = 0.05

plt.clf()
plt.plot([0,3],[0,3])
plt.xlabel('$\dot{\\theta}_{n}^+$')
plt.ylabel('$\dot{\\theta}_{n+1}^+$')

for t_touchdown in theta_dot_at_touchdown:
    x = t_touchdown[0:-2]
    y = t_touchdown[1:-1]

    for kk, (xx, yy) in enumerate(zip(x, y)):
        plt.title(kk)
        if kk > 0:
            plt.plot(x[0:kk], y[0:kk], 'ko')

        plt.plot(xx, yy, 'or')
        plt.pause(tstep)
        plt.plot([xx, yy], [yy, yy], '-', color = [dk, dk, dk])
        plt.pause(tstep)
        plt.plot([yy, yy], [yy, yy], '-', color = [dk, dk, dk])
        plt.pause(tstep)
        
        if kk+1 < len(x):
            plt.plot([yy, yy], [x[kk+1], y[kk+1]], '-', color = [dk, dk, dk])
            plt.pause(tstep)

# plt.axis([-1, 1, -5, 5])

plt.plot(x,y, '.')
plt.plot([-1,3],[-1,3])

plt.xlabel('$\dot{\\theta}_{n}^+$')
plt.ylabel('$\dot{\\theta}_{n+1}^+$')
plt.gcf()




<Figure size 846x587 with 1 Axes>

Let's lastly try to plot the theory prediction

In [34]:

gamma = np.pi/12
alpha = np.pi/6

random_state = np.array([1, 5])*(np.random.rand(100,2))
plt.clf()
dk = 0.9
alpha_viz = 0.1

theta_dot_at_touchdown = []

for rnd in random_state:
    t_data, y1, y2 = simulate_rimless(rnd, gamma, alpha)
    theta_dot_at_touchdown.append([y2[k][0] for k in range(1,len(y2))])

for t_touchdown in theta_dot_at_touchdown:
    x = t_touchdown[0:-2]
    y = t_touchdown[1:-1]
    plt.plot(x,y, '.k')
    
    
q_ = np.linspace(0,4,100)
q_plus1 = np.cos(2*alpha)*np.sqrt(q_**2 + 4 * (gravity/L)*np.sin(alpha)*np.sin(gamma))

plt.plot(q_, q_plus1, '--')
                        
    
plt.plot([0,3],[0,3])

plt.xlabel('$\dot{\\theta}_{n}^+$')
plt.ylabel('$\dot{\\theta}_{n+1}^+$')
plt.gcf()

<Figure size 846x587 with 1 Axes>

## Fin

So that is it. The rimless wheel shows a very nice and structured behavior, settling into walking dynamics that are independent of initial condition (there is some subtlety to this that I won't address here but can be read about [here](https://www.tandfonline.com/doi/abs/10.1080/14689360903429238)).

